In [1]:
import ROOT
import os
import matplotlib.pyplot as plt
import csv
import numpy as np

Welcome to JupyROOT 6.24/08


In [2]:
from simplified_simulation.convert_to_csv import get_layer_and_module
from simplified_simulation.convert_to_csv import get_not_close_to_beamline_stave
from simplified_simulation.convert_to_csv import get_close_to_beamline_stave

In [41]:
folder = '/nfs/dust/luxe/user/spatarod/Full_LUXE/background/event_1'
event_folder = os.listdir(folder)

output_name = f'e0ppw_{folder.split("/")[-3]}_000{folder.split("/")[-1].split("_")[-1]}_fl.csv'
print(f'{folder}/{output_name}')

/nfs/dust/luxe/user/spatarod/Full_LUXE/background/event_1/e0ppw_Full_LUXE_0001_fl.csv


In [42]:
x = []
y = []
z = []
hit_ID = []
layer_ID = []
module_ID = []
is_signal = []
particle_ID = []
energy = []

for file in event_folder:
    if not '.root' in file:
        continue
    in_file = ROOT.TFile.Open(f'{folder}/{file}',"READ")
    try:
        _ =  in_file.clusters
    except AttributeError:
        continue
    
    for i, event in enumerate(in_file.clusters):
        event.GetEntry(i)
        try:
            x_i = 1e-3 * event.tru_hit[0][0]
            y_i = 1e-3 * event.tru_hit[0][1]
            z_i = np.round(1e-3 * event.tru_hit[0][2], 3)
            
            x.append(x_i)
            y.append(y_i)
            z.append(z_i)
            layer, module = get_layer_and_module(x_i, y_i, z_i)
            layer_ID.append(layer)
            module_ID.append(module)
            particle_ID.append(event.tru_trackId[0])
            energy.append(event.tru_p[0][3])
            if event.isSignal == 1:
                is_signal.append(True)
            else:
                is_signal.append(False)
        except IndexError:            
            pass

with open(f'{folder}/{output_name}', 'w', newline='') as csv_file:
    fieldnames = ['hit_ID', 'x', 'y', 'z', 'layer_ID', 'module_ID', 'is_signal', 'particle_ID', 'particle_energy']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for i in range(len(x)):
        writer.writerow({'hit_ID': str(i),
                         'x': x[i],
                         'y': y[i],
                         'z': z[i],
                         'layer_ID': layer_ID[i],
                         'module_ID': module_ID[i],
                         'is_signal': is_signal[i],
                         'particle_ID': particle_ID[i],
                         'particle_energy': energy[i]})

In [ ]:
fig = plt.figure(figsize=(12,12), dpi=500)
ax = fig.add_subplot(projection='3d')
ax.scatter(z, x, y, s=0.7, label="$e^-$- laser background") # switch axes to match LUXE geometry description
ax.set_xlabel('z [m]', fontsize=16)
ax.set_ylabel('x [m]', fontsize=16)
ax.set_zlabel('y [m]', fontsize=16)
ax.tick_params(axis="x", labelsize=16)
ax.tick_params(axis="y", labelsize=16)
ax.tick_params(axis="z", labelsize=16)
ax.xaxis.labelpad=15
ax.yaxis.labelpad=15
ax.zaxis.labelpad=15
ax.legend(loc='best', fontsize=16)
ax.view_init(20, -130)


In [ ]:
print(len(x))

In [ ]:
plt.hist(energy)

In [ ]:
plt.hist(energy)